In [2]:
import matplotlib
matplotlib.use('nbagg')
import sys
import matplotlib.animation as anm
import matplotlib.pyplot as plt
import math
import matplotlib.patches as patches
import numpy as np
from enum import Enum


In [8]:
import sys                       
sys.path.append('../scripts/')
from scipy.stats import expon, norm, uniform
from enum import Enum
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import r2_score

In [77]:
class Mode(Enum):
    STATE_TRANSITION = 1
    STRAIGHT_TRANSITION = 2
    SHIFT_TRANSITION = 3

class World:
    def __init__(self, time_span, time_interval, debug=True):
        self.objects = []  
        self.debug = debug
        self.time_span = time_span  
        self.time_interval = time_interval 

        
    def append(self,obj):  
        self.objects.append(obj)
    
    def draw(self): 
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_subplot(111)
        ax.set_aspect('equal')             
        ax.set_xlim(-250,250)                  
        ax.set_ylim(-250,250)
        #ax.set_xlim(-5,5)
        #ax.set_ylim(-5,5)
        ax.set_xlabel("X",fontsize=10)                 
        ax.set_ylabel("Y",fontsize=10)                 
        elems = []
        
        if self.debug:        
            for i in range(int(self.time_span/self.time_interval)): self.one_step(i, elems, ax)
        else:
            self.ani = anm.FuncAnimation(fig, self.one_step, fargs=(elems, ax),
                                     frames=int(self.time_span/self.time_interval)+1, interval=int(self.time_interval*1000), repeat=False)
            plt.show()
        
    def one_step(self, i, elems, ax):
        while elems: elems.pop().remove()
        time_str = "t = %.2f[s]" % (self.time_interval*i)
        elems.append(ax.text(-245.4, 220, time_str, fontsize=10))
        #elems.append(ax.text(-4.4, 4.5, time_str,fontsize=10))
        for obj in self.objects:
            obj.draw(ax, elems)
            if hasattr(obj, "one_step"): obj.one_step(self.time_interval)    

class rob:
    def __init__(self,pose,agent):
        self.pose = pose
        self.agent = agent
        
        self.obs_stuck = 0
        self.obs_sign = 0
        
        self.angle = 3
        
        self.a = 0
        self.b = 0
    
    def draw(self,ax, elems):
        pass
    
    def state_transition(self,nu,omega,time,pose):
        t0 = pose[2]
        if math.fabs(omega) < 1e-10:
            return pose + np.array( [nu*math.cos(t0), 
                                     nu*math.sin(t0),
                                     omega ] ) * time
        else:
            return pose + np.array( [nu/omega*(math.sin(t0 + omega*time) - math.sin(t0)), 
                                     nu/omega*(-math.cos(t0 + omega*time) + math.cos(t0)),
                                     omega*time ] )
    #agentXクラスのもとでつかうことを想定          
    def transition(self, nu,omega, time,obs):
        if self.agent.mode == Mode.STATE_TRANSITION:
            return self.state_transition(nu,omega,time,self.pose)
        elif self.agent.mode == Mode.STRAIGHT_TRANSITION:
            return self.straight_transition(nu,omega, time, obs)
        elif self.agent.mode == Mode.SHIFT_TRANSITION:
            return self.shift_transition(nu,omega,time, obs)
        else:
            return self.state_transition(nu,omega,time, self.pose)
        
    def sensor_return(self,obs):
        if obs:         
            if np.abs(obs[0][0][1]) > self.angle /180 * math.pi :
                self.obs_stuck = np.abs(obs[0][0][1]) - self.angle / 180 * math.pi
                self.obs_sign = np.sign(obs[0][0][1])
                return  self.obs_sign * self.angle / 180 * math.pi
            else:
                return obs[0][0][1]

        if self.obs_stuck > self.angle / 180 * math.pi:
            self.obs_stuck = self.obs_stuck - self.angle / 180 * math.pi
            return self.obs_sign *  self.angle / 180 * math.pi
        else:
            self.a = self.obs_stuck
            self.b = self.obs_sign
            self.obs_stuck = 0
            self.obs_sign = 0
            return self.a *self.b / 180 * math.pi
        
   #agentXクラスのもとでつかうことを想定                         
    def straight_transition(self,nu,omega,time,obs):
        if obs:  
            if obs[0][0][0] < self.agent.distance_minimum:
                self.agent.decelerate_nu()
                self.agent.keep_straight_change()
                

        t0 = self.pose[2]        
            
        if math.fabs(omega) < 1e-10:
            return self.pose + np.array( [nu* math.cos(t0)*time, 
                             nu *math.sin(t0)*time,
                             omega*time  + self.sensor_return(obs)] )
        else:
            return self.pose + np.array( [nu  / omega*(math.sin(t0 + omega*time) - math.sin(t0)), 
                             nu  / omega*(-math.cos(t0 + omega*time) + math.cos(t0)),
                             omega*time + self.sensor_return(obs) ] )
        
    #agentXクラスのもとでつかうことを想定                      
    def shift_transition(self, nu, omega, time, obs):
        if obs:
            if obs[0][0][0] < self.agent.distance_minimum:
                self.agent.decelerate_nu
                self.agent.keep_shift_change()
            
        t0 = self.pose[2]
        
        if math.fabs(omega) < 1e-10:
            return self.pose + np.array( [nu *math.cos(t0)*time+2.0, 
                                 nu *math.sin(t0)*time,
                                 omega*time+self.sensor_return(obs) ] )/ math.sqrt((nu * self.accelerate_rate *math.cos(t0)*time+2.0)**2 +(nu * self.accelerate_rate *math.sin(t0)*time)**2) * math.sqrt((nu * self.accelerate_rate *math.cos(t0)*time)**2 +(nu * self.accelerate_rate *math.sin(t0)*time)**2) 
        else:
            return self.pose + np.array( [nu/omega*(math.sin(t0 + omega*time) - math.sin(t0))+2.0, 
                                 nu  /omega*(-math.cos(t0 + omega*time) + math.cos(t0)),
                                 omega*time+self.sensor_return(obs) ] )/ math.sqrt((nu/omega*(math.sin(t0 + omega*time) - math.sin(t0))+2.0)**2 + (nu* self.accelerate_rate/omega*(-math.cos(t0 + omega*time) + math.cos(t0))**2 )) * math.sqrt((nu* self.accelerate_rate/omega*(math.sin(t0 + omega*time) - math.sin(t0)))**2 + (nu* self.accelerate_rate/omega*(-math.cos(t0 + omega*time) + math.cos(t0))**2 ))

# In[3]:

class IdealRobot(rob):   
    def __init__(self, pose, agent=None, sensor=None, color="black"):    # 引数を追加
        super().__init__(pose,agent)
        self.pose = pose
        self.r = 10  
        self.color = color 
        self.agent = agent
        self.poses = [pose]
        self.id = 0
        self.sensor = sensor    # 追加
    
    def draw(self, ax, elems):         ### call_agent_draw
        x, y, theta = self.pose  
        xn = x + self.r * math.cos(theta)
        yn = y + self.r * math.sin(theta)
        elems += ax.plot([x,xn], [y,yn], color=self.color)
        c = patches.Circle(xy=(x, y), radius=self.r, fill=False, color=self.color) 
        elems.append(ax.add_patch(c))
        self.poses.append(self.pose)
        elems.append(ax.text(self.pose[0]-20, self.pose[1]-20, "child AUV" + str(self.id), fontsize=8))         
        elems += ax.plot([e[0] for e in self.poses], [e[1] for e in self.poses], linewidth=0.5, color="black")
        if self.sensor and len(self.poses) > 1:
            self.sensor.draw(ax, elems, self.poses[-2])
        if self.agent and hasattr(self.agent, "draw"):                               #以下2行追加   
            self.agent.draw(ax, elems)

    def one_step(self, time_interval):
        if not self.agent: return        
        obs =self.sensor.data(self.pose) if self.sensor else None #追加
        nu, omega = self.agent.decision(obs) #引数追加
        self.pose = self.state_transition(nu, omega, time_interval, self.pose)
        if self.sensor: self.sensor.data(self.pose)   
            
# In[4]:


class Agent: 
    def __init__(self, nu, omega):
        self.nu = nu
        self.omega = omega
        self.mode = Mode.STATE_TRANSITION
        
    def decision(self, observation=None):
        return self.nu, self.omega

class AgentX:
    def __init__(self, nu, omega,accelerate_rate, distance_minimum,distance_maximum):
        self.nu = nu
        self.omega = omega
        
        self.mode = Mode.STATE_TRANSITION
        
        self.accelerate_rate = accelerate_rate
        self.shift_switch = False
        self.keep_straight = False
        self.keep_shift = False
        
        self.distance_maximum = distance_maximum
        self.distance_minimum = distance_minimum
        self.count = 0
        
        self.sensor_num = 0
        self.sensor_stuck = []
        self.sensor_stuck_train = []
        self.sensor_stuck_test = []
        
        self.t_train = []
        self.t_test = []
        
        if(distance_minimum > distance_maximum):
            print("input error")
            sys.exit()
            
        
    def decision(self, observation=None):
        if observation:
            self.mode = self.mode_change(observation)
        return self.nu, self.omega
    
    def shift_switch_change(self):
        self.shift_switch = not self.shift_switch
    
    def keep_straight_change(self):
        self.keep_straight = not self.keep_straight
    
    def keep_shift_change(self):
        self.keep_shift = not self.keep_shift
        
    def accelerate_nu(self):
        self.nu = self.nu * self.accelerate_rate
    
    def decelerate_nu(self):
        self.nu = self.nu / self.accelerate_rate
        
    def mode_change(self,obs):
        if ((obs[0][0][0] > self.distance_maximum) and(self.shift_switch == False)) or (self.keep_straight == True) :
            if (self.keep_straight == False) :
                self.accelerate_nu()
                self.keep_straight_change()
            return Mode.STRAIGHT_TRANSITION
        elif ((obs[0][0][0] > self.distance_maximum) and (self.shift_switch == True)) or (self.keep_shift == True):
            if (self.keep_shift == False):
                self.accelerate_nu()
                self.nu = self.nu * self.accelerate_rate
            return Mode.SHIFT_TRANSITION
        else:
            return Mode.STATE_TRANSITION
        
    def ligression(self,obs, time_interval):
        train_length = 15
        test_length = 3

        lr = LinearRegression()
        self.count = self.count + time_interval
#         for i in range(1,length ):
#             if i < train_length / 3 + 1:
#                 a = t[i - 1] * 2 + 3
#                 sensor_stuck.append(a)
#                 sensor_stuck_train.append(a)
#                 t[i] = t[i-1] + dt
#                 t_train.append(t[i-1])

#             elif i < train_length / 3 + test_length + 1:
#                 a = t[i - 1]  * 2 + 3
#                 sensor_stuck.append(a)
#                 sensor_stuck_test.append(a)
#                 t[i] = t[i-1] + dt
#                 t_test.append(t[i-1])


#             elif i< train_length + test_length + 1:
#                 a = t[i - 1]  * 2 + 3
#                 sensor_stuck.append(a)
#                 sensor_stuck_train.append(sensor_stuck_test[0])
#                 sensor_stuck_test.pop(0)
#                 sensor_stuck_test.append(a)

#                 t[i] = t[i-1] + dt
#                 t_train.append(t_test[0])
#                 t_test.pop(0)
#                 t_test.append(t[i-1])

#             elif i< train_length + test_length + train_length / 3 + 1: 
#                 a = t[i - 1]  * 2 + 3
#                 sensor_stuck.append(a)
#                 sensor_stuck_train.append(sensor_stuck_test[0])
#                 sensor_stuck_test.pop(0)
#                 sensor_stuck_test.append(a)

#                 t[i] = t[i-1] + dt
#                 t_train.append(t_test[0])
#                 t_test.pop(0)
#                 t_test.append(t[i-1])        

#             else:
#                 a = t[i-1] * 2 + 3
#                 sensor_stuck.append(a)
#                 sensor_stuck_train.pop(0)
#                 sensor_stuck_train.append(sensor_stuck_test[0])
#                 sensor_stuck_test.pop(0)
#                 sensor_stuck_test.append(a)

#                 t[i] = t[i-1]  + dt
#                 t_train.pop(0)
#                 t_train.append(t_test[0])
#                 t_test.pop(0)
#                 t_test.append(t[i-1])

#         print(t_test)
#         print(sensor_stuck_test)
#         print(t_train)
#         print(sensor_stuck_train)

        if obs:
            if self.sensor_num < train_length / 3 + 1:
                self.sensor_stuck.append(obs[0][0][0])
                self.sensor_stuck_train.append(obs[0][0][0])
                self.t_train.append(self.count)
                self.sensor_num = self.sensor_num + 1
#                 print('1')
                
            elif self.sensor_num < train_length / 3 + test_length + 1:
                self.sensor_stuck.append(obs[0][0][0])
                self.sensor_stuck_test.append(obs[0][0][0])
                self.t_test.append(self.count)
                self.sensor_num = self.sensor_num + 1
                print('2')
                
            elif self.sensor_num < train_length + test_length + 1:
                self.sensor_stuck.append(obs[0][0][0])
                self.sensor_stuck_train.append(self.sensor_stuck_test[0])
                self.sensor_stuck_test.pop(0)
                self.sensor_stuck_test.append(obs[0][0][0])
                
                self.t_train.append(self.t_test[0])
                self.t_test.pop(0)
                self.t_test.append(self.count)
                self.sensor_num = self.sensor_num + 1
                print('3')
            else:
                self.sensor_stuck.append(obs[0][0][0])
                self.sensor_stuck_train.pop(0)
                self.sensor_stuck_train.append(self.sensor_stuck_test[0])
                self.sensor_stuck_test.pop(0)
                self.sensor_stuck_test.append(obs[0][0][0])

                self.t_train.pop(0)
                self.t_train.append(self.t_test[0])
                self.t_test.pop(0)
                self.t_test.append(self.count)
                self.sensor_num = self.sensor_num + 1
                print('4')
                
        if (obs != None) and (len(self.sensor_stuck_train) > 1) and (len(self.sensor_stuck_test) > 1 )  and (len(self.t_train) > 1 ) and (len(self.t_test) > 1 ):
            print(obs)
            print("t_train length:", len(self.t_train))
#             print(self.sensor_stuck_train)
            X = np.array(self.t_train).reshape(-1, 1)
            y = np.array(self.sensor_stuck_train).reshape(-1, 1)

            X_test = np.array(self.t_test).reshape(-1,1)

            lr = LinearRegression()
            lr.fit(X, y)
            y_train_pred = lr.predict(X)
            y_test_pred = lr.predict(X_test)
            y_test = np.array(self.sensor_stuck_test).reshape(-1,1)
            
#             print(f"y = {lr.intercept_[0]:.2f} + {lr.coef_[0][0]:.2f}x")
            print("y_test: ", y_test)
            print("y_test_pred: ", y_test_pred)


            print(r2_score(y, y_train_pred),r2_score(y_test, y_test_pred))

        
class AgentY:
    def __init__(self,time_interval, data):
        self.nu = 0
        self.omega = 0
        self.time_interval = time_interval
        self.data = data
        self.iterator = iter(self.data)
        self.current_data = []
        self.time = 0
        self.mode = Mode.STATE_TRANSITION
        
    def decision(self, observation=None):
        self.data_change()
        return self.nu, self.omega
        
    def data_change(self):
        if(self.time <= 1e-10):
            self.current_data = next(self.iterator, "end")
            if(self.current_data == "end"):
#                 print("byebye")
                return
#             print(self.current_data)
            self.nu = self.current_data[0]
            self.omega = self.current_data[1]
            self.time = self.current_data[2]
        self.time = self.time - self.time_interval
#        print(self.time)
            
# In[5]:


class Landmark:
    def __init__(self, pos , agent = None):
        self.pos = pos
        self.agent = agent
        self.id = None
        
    def draw(self, ax, elems):
        c = ax.scatter(self.pos[0], self.pos[1], s=30, marker="*", label="landmarks", color="orange")
        elems.append(c)
#         elems.append(ax.text(self.pos[0], self.pos[1], "id:" + str(self.id), fontsize=10))
        elems.append(ax.text(self.pos[0]-20, self.pos[1]-20, "parent AUV" + str(self.id), fontsize=8))
    def state_transition(cls, nu, omega, time, pose):
        t0 = pose[2]
        if math.fabs(omega) < 1e-10:
            return pose + np.array( [nu*math.cos(t0), 
                                     nu*math.sin(t0),
                                     omega ] ) * time
        else:
            return pose + np.array( [nu/omega*(math.sin(t0 + omega*time) - math.sin(t0)), 
                                     nu/omega*(-math.cos(t0 + omega*time) + math.cos(t0)),
                                     omega*time ] )

    def one_step(self, time_interval):
        if not self.agent: return        
        nu, omega = self.agent.decision() #引数追加
        self.pos = self.state_transition(nu, omega, time_interval, self.pos) 


# In[6]:


class Map:
    def __init__(self):       # 空のランドマークのリストを準備
        self.objects = []
        
    def append_object(self, ob):       # ランドマークを追加
        ob.id = len(self.objects)           # 追加するランドマークにIDを与える
        self.objects.append(ob)

    def draw(self, ax, elems):                 # 描画（Landmarkのdrawを順に呼び出し）
        for ob in self.objects: ob.draw(ax, elems)

    def one_step(self,time_interval):
        for ob in self.objects: ob.one_step(time_interval)
                         


# In[7]:


class IdealCamera:
    def __init__(self, env_map, distance_range=(0.5, 350), direction_range=(-math.pi, math.pi)):
        self.map = env_map
        self.lastdata = []
        
        self.distance_range = distance_range
        self.direction_range = direction_range
        
    def visible(self, polarpos):  # ランドマークが計測できる条件
        if polarpos is None:
            return False
        #print(polarpos[0])
        #print(self.distance_range[0])
        #print(self.distance_range[1])
        #print(self.distance_range[0] <= polarpos[0] and polarpos[0] <= self.distance_range[1])
    
        return self.distance_range[0] <= polarpos[0] <= self.distance_range[1]                 and self.direction_range[0] <= polarpos[1] <= self.direction_range[1]
        
    def data(self, cam_pose):
        observed = []
        for lm in self.map.objects:
            z = self.observation_function(cam_pose, lm.pos)
            if self.visible(z):               # 条件を追加
                observed.append((z, lm.id))   # インデント
            
        self.lastdata = observed
        return observed
    
    @classmethod
    def observation_function(cls, cam_pose, obj_pos):
        #print("obj=",obj_pos)
        #print("cam=",cam_pose)
        diff = obj_pos[0:2]- cam_pose[0:2]
        phi = math.atan2(diff[1], diff[0]) - cam_pose[2]
        while phi >= np.pi: phi -= 2*np.pi
        while phi < -np.pi: phi += 2*np.pi
        return np.array( [np.hypot(*diff), phi ] ).T
    
    def draw(self, ax, elems, cam_pose): 
        for lm in self.lastdata:
            x, y, theta = cam_pose
            distance, direction = lm[0][0], lm[0][1]
            lx = x + distance * math.cos(direction + theta)
            ly = y + distance * math.sin(direction + theta)
            elems += ax.plot([x,lx], [y,ly], color="pink")


In [78]:

# In[2]:


class Robot(IdealRobot):
        
    def __init__(self, pose, agent=None, sensor=None, color="black",                            noise_per_meter=5, noise_std=math.pi/180, bias_rate_stds=(0.1,0.1),                            expected_stuck_time=1e100, expected_escape_time = 1e-100,                           expected_kidnap_time=1e100, kidnap_range_x = (-5.0,5.0), kidnap_range_y = (-5.0,5.0)): #追加
        super().__init__(pose, agent, sensor, color)
        #noise に必要な変数
        #大域でグラフを生成しないと関数の呼び出しごとに関数を生成することになる
        #出てくる値がランダムじゃなくなる
        self.noise_pdf = expon(scale=1.0/(1e-100 + noise_per_meter))
        self.distance_until_noise = self.noise_pdf.rvs()
        self.theta_noise = norm(scale=noise_std)
        
        #biasに必要な関数
        self.bias_rate_nu = norm.rvs(loc=1.0, scale=bias_rate_stds[0])
        self.bias_rate_omega = norm.rvs(loc=1.0, scale=bias_rate_stds[1]) 
        
        #stuckに必要な関数
        self.stuck_pdf = expon(scale=expected_stuck_time) 
        self.escape_pdf = expon(scale=expected_escape_time)
        self.is_stuck = False
        self.time_until_stuck = self.stuck_pdf.rvs()
        self.time_until_escape = self.escape_pdf.rvs()
        
        #kidnapに必要な関数
        self.kidnap_pdf = expon(scale=expected_kidnap_time) 
        self.time_until_kidnap = self.kidnap_pdf.rvs()
        rx, ry = kidnap_range_x, kidnap_range_y
        self.kidnap_dist = uniform(loc=(rx[0], ry[0], 0.0), scale=(rx[1]-rx[0], ry[1]-ry[0], 2*math.pi ))
        
        #one_stepに必要な関数
        self.is_first = True
        
        self.const_time = 1.0
        self.sensor_time = self.const_time 
        
        self.sensor_stuck = []
        
        
    def noise(self, pose, nu, omega, time_interval):
        self.distance_until_noise -= abs(nu)*time_interval + self.r*abs(omega)*time_interval
        if self.distance_until_noise <= 0.0:
            self.distance_until_noise += self.noise_pdf.rvs()
            pose[2] += self.theta_noise.rvs()
            
        return pose
        
    def bias(self, nu, omega): 
        return nu*self.bias_rate_nu, omega*self.bias_rate_omega
    
    def stuck(self, nu, omega, time_interval):
        if self.is_stuck:
            self.time_until_escape -= time_interval
            if self.time_until_escape <= 0.0:
                self.time_until_escape += self.escape_pdf.rvs()
                self.is_stuck = False
        else:            
            self.time_until_stuck -= time_interval
            if self.time_until_stuck <= 0.0:
                self.time_until_stuck += self.stuck_pdf.rvs()
                self.is_stuck = True

        return nu*(not self.is_stuck), omega*(not self.is_stuck)
         
    
    def kidnap(self, pose, time_interval):
        self.time_until_kidnap -= time_interval
        if self.time_until_kidnap <= 0.0:
            self.time_until_kidnap += self.kidnap_pdf.rvs()
            return np.array(self.kidnap_dist.rvs()).T
        else:
            return pose
        
#     def sensor_append(self, mode, obs):
#         if mode == True:
#             if obs:
#                 self.sensor_stuck.append(obs)
    
    
    def sensor_count(self, time_interval):
        if(self.sensor_time >= 0.1) and (self.is_first == False):
            self.sensor_time -= time_interval
            return None
        elif self.is_first == True:
            self.is_first = False
            return self.sensor.data(self.pose) if self.sensor else None
        else:
            self.sensor_time = self.const_time
            return self.sensor.data(self.pose) if self.sensor else None
        
    def one_step(self,time_interval):
        if not self.agent: return
        
        obs = self.sensor_count(time_interval)
            
#         self.sensor_append(True,obs)

        
        self.agent.ligression(obs,time_interval)
        
        nu, omega = self.agent.decision(obs)
        nu, omega = self.bias(nu,omega)
        nu, omega = self.stuck(nu,omega,time_interval)

        
        self.pose = self.transition(nu,omega,time_interval,obs)

        self.pose = self.noise(self.pose, nu,omega, time_interval)
        self.pose = self.kidnap(self.pose, time_interval)


# In[3]:


class Camera(IdealCamera): ###noisesim_occlusion### 
    def __init__(self, env_map,
                 distance_range=(0.5, 350),
                 direction_range=(-math.pi, math.pi),
                 distance_noise_rate=0.05, direction_noise=math.pi/180,
                 distance_bias_rate_stddev=0.05, direction_bias_stddev=math.pi/180,
                 phantom_prob=0.0, phantom_range_x=(-5.0,5.0), phantom_range_y=(-5.0,5.0),
                 oversight_prob=0.1, occlusion_prob=0.0): #occlusion_prob追加
        super().__init__(env_map, distance_range, direction_range)
        
        self.distance_noise_rate = distance_noise_rate
        self.direction_noise = direction_noise  
        self.distance_bias_rate_std = norm.rvs(scale=distance_bias_rate_stddev)
        self.direction_bias = norm.rvs(scale=direction_bias_stddev) 
        
        rx, ry = phantom_range_x, phantom_range_y
        self.phantom_dist = uniform(loc=(rx[0], ry[0]), scale=(rx[1]-rx[0], ry[1]-ry[0]))
        self.phantom_prob = phantom_prob
        
        self.oversight_prob = oversight_prob
        self.occlusion_prob = occlusion_prob #追加
        
    def noise(self, relpos):  
        ell = norm.rvs(loc=relpos[0], scale=relpos[0]*self.distance_noise_rate)
        phi = norm.rvs(loc=relpos[1], scale=self.direction_noise)
        return np.array([ell, phi]).T
    
    def bias(self, relpos): 
        return relpos + np.array([relpos[0]*self.distance_bias_rate_std,
                                  self.direction_bias]).T
    
    def phantom(self, cam_pose, relpos):
        if uniform.rvs() < self.phantom_prob:
            pos = np.array(self.phantom_dist.rvs()).T
            return self.observation_function(cam_pose, pos)
        else:
            return relpos
        
    def oversight(self, relpos):
        if uniform.rvs() < self.oversight_prob:
            return None
        else:
            return relpos
        
    def occlusion(self, relpos): #追加
        if uniform.rvs() < self.occlusion_prob:
            ell = relpos[0] + uniform.rvs()*(self.distance_range[1] - relpos[0])
            return np.array([ell, relpos[1]]).T   
        else:
            return relpos
    
    def data(self, cam_pose):
        observed = []
        for lm in self.map.objects:
            #print("camera data obj=",lm.pos)
            #print("camera data cam=", cam_pose)
            z = self.observation_function(cam_pose, lm.pos)
            z = self.phantom(cam_pose, z) 
            z = self.occlusion(z) #追加
            z = self.oversight(z)
            #print("a", self.visible(z))
            if self.visible(z):
                #print("b")
                z = self.bias(z)
                z = self.noise(z)  
                observed.append((z, lm.id))
                
            
        self.lastdata = observed 
        return observed

In [79]:
from scipy.stats import multivariate_normal
import random #追加
import copy

# In[2]:


class Particle(rob): 
    def __init__(self, init_pose, agent, weight ):
        super().__init__(init_pose, agent)
        self.weight = weight
        self.obs = []
        
    def motion_update(self, nu, omega, time, noise_rate_pdf): 
        ns = noise_rate_pdf.rvs()
        pnu = nu + ns[0]*math.sqrt(abs(nu)/time) + ns[1]*math.sqrt(abs(omega)/time)
        pomega = omega + ns[2]*math.sqrt(abs(nu)/time) + ns[3]*math.sqrt(abs(omega)/time)
        if self.obs:
            self.mode = self.agent.mode_change(self.obs)
        self.pose = self.transition(pnu, pomega, time, self.obs)
        
    def observation_update(self, observation, envmap, distance_dev_rate, direction_dev):  #変更_
        for d in observation:
            obs_pos = d[0]
            obs_id = d[1]
            
            ##パーティクルの位置と地図からランドマークの距離と方角を算出##
            pos_on_map = envmap.objects[obs_id].pos
            particle_suggest_pos = IdealCamera.observation_function(self.pose, pos_on_map)
            
            ##尤度の計算##
            distance_dev = distance_dev_rate*particle_suggest_pos[0]
            cov = np.diag(np.array([distance_dev**2, direction_dev**2]))
            self.weight *= multivariate_normal(mean=particle_suggest_pos, cov=cov).pdf(obs_pos)


# In[3]:


class Mcl:    ###mlparticle（12〜18行目）
    def __init__(self, envmap, init_pose, num,accelerate_rate,distance_minimum, distance_maximum,  motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2},                  distance_dev_rate=0.14, direction_dev=0.05):
        self.accelerate_rate = accelerate_rate
        self.distance_maximum = distance_minimum
        self.distance_minimum = distance_maximum
        self.particles = [Particle(init_pose,AgentX(0,0, accelerate_rate, distance_minimum,distance_maximum), 1.0/num) for i in range(num)]
        self.map = envmap
        self.distance_dev_rate = distance_dev_rate
        self.direction_dev = direction_dev

        v = motion_noise_stds
        c = np.diag([v["nn"]**2, v["no"]**2, v["on"]**2, v["oo"]**2])
        self.motion_noise_rate_pdf = multivariate_normal(cov=c)
        self.ml = self.particles[0] #追加
        self.pose = self.ml.pose #追加（互換性のため）
        
        self.sigma_xs = []
        self.sigma_ys = []
        
    def set_ml(self): #追加
        i = np.argmax([p.weight for p in self.particles])
        self.ml = self.particles[i]
        self.pose = self.ml.pose
        
    def motion_update(self, nu, omega, time): 
        x_particle = []
        y_particle = []
        for p in self.particles: 
            p.motion_update(nu, omega, time, self.motion_noise_rate_pdf)
            x_particle.append(p.pose[0])
            y_particle.append(p.pose[1])
        self.sigma_xs.append(np.std(x_particle))
        self.sigma_ys.append(np.std(y_particle))
            
    def observation_update(self, observation): 
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
        self.set_ml() #リサンプリング前に実行
        self.resampling() 
            
    def resampling(self): ###systematicsampling
        ws = np.cumsum([e.weight for e in self.particles]) #重みを累積して足していく（最後の要素が重みの合計になる）
        if ws[-1] < 1e-100: ws = [e + 1e-100 for e in ws]  #重みの合計が0のときの処理
            
        step = ws[-1]/len(self.particles)   #正規化されていない場合はステップが「重みの合計値/N」になる
        r = np.random.uniform(0.0, step)
        cur_pos = 0
        ps = []            #抽出するパーティクルのリスト
        
        while(len(ps) < len(self.particles)):
            if r < ws[cur_pos]:
                ps.append(self.particles[cur_pos])  #もしかしたらcur_posがはみ出るかもしれませんが例外処理は割愛で
                r += step
            else:
                cur_pos += 1

        self.particles = [copy.deepcopy(e) for e in ps]                   #以下の処理は前の実装と同じ
        for p in self.particles: p.weight = 1.0/len(self.particles)
        
    def draw(self, ax, elems):  
        xs = [p.pose[0] for p in self.particles]
        ys = [p.pose[1] for p in self.particles]
        vxs = [math.cos(p.pose[2])*p.weight*len(self.particles) * 800 for p in self.particles] #重みを要素に反映
        vys = [math.sin(p.pose[2])*p.weight*len(self.particles) * 800 for p in self.particles]  #重みを要素に反映
        elems.append(ax.quiver(xs, ys, vxs, vys,                                angles='xy', scale_units='xy',color="orange", alpha=0.5)) #変更


# In[4]:


class EstimationAgent(AgentX): 
    def __init__(self, time_interval, nu, omega,accelerate_rate,distance_minimum, distance_maximum, estimator):
        super().__init__(nu, omega,accelerate_rate,distance_minimum, distance_maximum)
        self.estimator = estimator
        self.time_interval = time_interval
        
        self.prev_nu = 0.0
        self.prev_omega = 0.0
        
#         self.sensor_time = 1
#         self.is_first = True
        
    def decision(self, observation=None): 
        self.estimator.motion_update(self.prev_nu, self.prev_omega, self.time_interval)
        self.prev_nu, self.prev_omega = self.nu, self.omega
        if observation:
            self.estimator.observation_update(observation)
            self.mode = self.mode_change(observation)
        return self.nu, self.omega
        
    def draw(self, ax, elems): ###mlwrite
        self.estimator.draw(ax, elems)
        x, y, t = self.estimator.pose #以下追加
        s = "({:.2f}, {:.2f}, {})".format(x,y,int(t*180/math.pi)%360)
        elems.append(ax.text(0, 220,s, fontsize=8))

In [80]:
time_interval = 0.1
world = World(130, time_interval, debug = True)

m = Map()
data1 = [[3.0,0,25],[1.0, -20.0 * math.pi / 180, 5.5],[7.0,0,30],[0, -20.0 * math.pi / 180, 5.5],[7.0,0,30],[1.0, -20.0 * math.pi / 180, 5.5],[7.0,0,30],]
agent1 = AgentY(0.1,data1)
initial_pose_1 = np.array([0,100,0]).T
m.append_object(Landmark(initial_pose_1, agent = agent1))
world.append(m)    

m1 = Map()
initial_pose_2 = np.array([-100,50,0]).T
initial_pose_3 = np.array([-100,150,0]).T
accelerate_rate = 5
distance_minimum = 150
distance_maximum = 175
estimator2 = Mcl(m,initial_pose_2,100, accelerate_rate,distance_minimum, distance_maximum)
estimator3 = Mcl(m,initial_pose_3,100, accelerate_rate,distance_minimum, distance_maximum)
c= EstimationAgent(time_interval,2,0,accelerate_rate,distance_minimum, distance_maximum, estimator3)
b = EstimationAgent(time_interval,2,0,accelerate_rate,distance_minimum, distance_maximum, estimator2)
r_2 = Robot(initial_pose_2 , sensor = Camera(m),agent = b, color = "blue")
r_1 = Robot(initial_pose_3,sensor = Camera(m), agent = c)
m1.append_object(r_2)
m1.append_object(r_1)
world.append(m1)
world.draw()

2
2
[(array([119.08478289,  -0.31310751]), 0)]
t_train length: 6
y_test:  [[102.24221665]
 [119.08478289]]
y_test_pred:  [[111.73904094]
 [113.30340505]]
0.35906329438571205 0.12847242574271023
2
2
[(array([108.76645696,  -0.35604778]), 0)]
t_train length: 6
y_test:  [[102.24221665]
 [119.08478289]
 [108.76645696]]
y_test_pred:  [[111.73904094]
 [113.30340505]
 [114.86776916]]
0.35906329438571205 -0.11512316180796667
2
[(array([119.75677586,   0.57441895]), 0)]
t_train length: 6
y_test:  [[117.0295253 ]
 [119.75677586]]
y_test_pred:  [[126.20620031]
 [128.28723194]]
0.3510347016793154 -41.21087617863835
[]
t_train length: 6
y_test:  [[102.24221665]
 [119.08478289]
 [108.76645696]]
y_test_pred:  [[111.73904094]
 [113.30340505]
 [114.86776916]]
0.35906329438571205 -0.11512316180796667
2
[(array([119.50433869,   0.7192435 ]), 0)]
t_train length: 6
y_test:  [[117.0295253 ]
 [119.75677586]
 [119.50433869]]
y_test_pred:  [[126.20620031]
 [128.28723194]
 [130.36826357]]
0.3510347016793154 -59

4
[(array([129.83258655,   1.48660864]), 0)]
t_train length: 16
y_test:  [[123.87671427]
 [130.33096634]
 [129.83258655]]
y_test_pred:  [[136.13491415]
 [137.19547238]
 [138.25603061]]
0.6128954421412349 -9.403683420393767
4
[(array([116.36424925,  -0.77722213]), 0)]
t_train length: 16
y_test:  [[123.71046756]
 [135.95731835]
 [116.36424925]]
y_test_pred:  [[124.93146058]
 [125.49351234]
 [126.0555641 ]]
0.1934563630659365 -0.04571017207388661
4
[(array([132.03506073,   1.49307923]), 0)]
t_train length: 16
y_test:  [[130.33096634]
 [129.83258655]
 [132.03506073]]
y_test_pred:  [[133.78066335]
 [134.54957967]
 [135.318496  ]]
0.3777265125353634 -15.842504120638917
4
[(array([125.36887824,  -0.70555768]), 0)]
t_train length: 16
y_test:  [[135.95731835]
 [116.36424925]
 [125.36887824]]
y_test_pred:  [[124.03149243]
 [124.38191589]
 [124.73233935]]
0.09251492571271458 -0.0756449298959212
4
[(array([123.04474024,   1.48773021]), 0)]
t_train length: 16
y_test:  [[129.83258655]
 [132.03506073

4
[(array([37.81143908, -0.77226337]), 0)]
t_train length: 16
y_test:  [[35.63782045]
 [34.80115371]
 [37.81143908]]
y_test_pred:  [[18.13646476]
 [11.70448142]
 [ 5.27249807]]
0.9838546392169045 -392.16811963496
4
[(array([169.5349623,  -0.3485694]), 0)]
t_train length: 16
y_test:  [[183.52324582]
 [161.77052578]
 [169.5349623 ]]
y_test_pred:  [[164.84733088]
 [167.36686742]
 [169.88640395]]
0.6336759505844336 -0.5644430702457897
4
[(array([41.10806192, -0.81547101]), 0)]
t_train length: 16
y_test:  [[34.80115371]
 [37.81143908]
 [41.10806192]]
y_test_pred:  [[16.12210514]
 [ 9.9913552 ]
 [ 3.86060527]]
0.9697460438158526 -125.12855052399293
4
[(array([1.79015075e+02, 3.89732335e-03]), 0)]
t_train length: 16
y_test:  [[161.77052578]
 [169.5349623 ]
 [179.01507462]]
y_test_pred:  [[175.33494359]
 [178.49268105]
 [181.65041851]]
0.7429871579805213 -0.8178251144309865
4
[(array([43.59419334, -0.97303494]), 0)]
t_train length: 16
y_test:  [[37.81143908]
 [41.10806192]
 [43.59419334]]
y_te

4
[(array([86.92954194, -2.26442474]), 0)]
t_train length: 16
y_test:  [[83.75264954]
 [96.85681878]
 [86.92954194]]
y_test_pred:  [[100.04437917]
 [103.39984576]
 [106.75531234]]
0.8528854336941583 -6.504130604560159
4
[(array([106.46134528,  -0.28395616]), 0)]
t_train length: 16
y_test:  [[143.20160831]
 [112.28216028]
 [106.46134528]]
y_test_pred:  [[146.27452751]
 [141.43514272]
 [139.82201446]]
0.3416774692112373 -1.5288355388403199
4
[(array([107.34866643,  -2.33305169]), 0)]
t_train length: 16
y_test:  [[ 96.85681878]
 [ 86.92954194]
 [107.34866643]]
y_test_pred:  [[ 98.1978811 ]
 [101.10469436]
 [104.01150763]]
0.7934656369773874 -0.025640173937759192
4
[(array([98.96054954, -0.40875269]), 0)]
t_train length: 16
y_test:  [[112.28216028]
 [106.46134528]
 [ 98.96054954]]
y_test_pred:  [[138.46853216]
 [136.52976757]
 [134.59100297]]
0.44898751022022765 -31.05457041128207
4
[(array([109.4068529 ,  -2.39189116]), 0)]
t_train length: 16
y_test:  [[ 86.92954194]
 [107.34866643]
 [109

4
[(array([ 1.9256403e+02, -1.0811101e-01]), 0)]
t_train length: 16
y_test:  [[207.1683746 ]
 [193.47842875]
 [192.56402998]]
y_test_pred:  [[227.66887525]
 [235.16534589]
 [242.66181653]]
0.9598463045195089 -33.87487238898077
4
[(array([145.06002681,  -1.81677229]), 0)]
t_train length: 16
y_test:  [[134.91634703]
 [129.74249332]
 [145.06002681]]
y_test_pred:  [[114.48267607]
 [115.83656757]
 [117.19045908]]
0.3528195995356186 -10.427350069259473
4
[(array([ 1.65393574e+02, -2.22387168e-02]), 0)]
t_train length: 16
y_test:  [[193.47842875]
 [192.56402998]
 [165.39357414]]
y_test_pred:  [[231.78711048]
 [239.14764578]
 [246.50818108]]
0.9507753167955426 -19.062131546486313
4
[(array([161.8669835,  -1.8630781]), 0)]
t_train length: 16
y_test:  [[129.74249332]
 [145.06002681]
 [161.8669835 ]]
y_test_pred:  [[123.09010007]
 [125.24556892]
 [127.40103777]]
0.5816087675998549 -2.146573760830995
4
[(array([1.78017900e+02, 2.19686062e-02]), 0)]
t_train length: 16
y_test:  [[192.56402998]
 [165

4
[(array([109.38799878,  -1.08072533]), 0)]
t_train length: 16
y_test:  [[102.30306624]
 [111.01161612]
 [109.38799878]]
y_test_pred:  [[82.68641119]
 [76.9970356 ]
 [71.30766001]]
0.908815096360328 -68.75724262449981
4
[(array([142.61054882,  -2.4895305 ]), 0)]
t_train length: 16
y_test:  [[138.72363075]
 [143.40939877]
 [142.61054882]]
y_test_pred:  [[143.91564992]
 [142.81191646]
 [141.70818299]]
0.3851766105849328 -1.2381660373624932
4
[(array([112.95623868,  -1.14215371]), 0)]
t_train length: 16
y_test:  [[111.01161612]
 [109.38799878]
 [112.95623868]]
y_test_pred:  [[83.37478082]
 [78.25297102]
 [73.13116121]]
0.8832330316475892 -518.9817560041373
4
[(array([144.84272084,  -2.55368997]), 0)]
t_train length: 16
y_test:  [[143.40939877]
 [142.61054882]
 [144.84272084]]
y_test_pred:  [[138.64398319]
 [136.96277524]
 [135.28156728]]
0.7660912047186217 -56.075848092768716
4
[(array([116.08578839,  -1.21776697]), 0)]
t_train length: 16
y_test:  [[109.38799878]
 [112.95623868]
 [116.08